In [9]:
from datetime import datetime, timedelta

In [23]:
arcpy.env.workspace = r'C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\Vindefjallen_testing\Arc_Vindefjellen\Arc_Vindefjellen.gdb'

# Specify the feature class and field name
feature_class = 'Vindefjallen_VV_MDR_test'
field_name = 'dags'

print('Done')

Done


In [25]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
    arcpy.management.CreateMosaicDataset(
        in_workspace=r"C:\Users\trygg\Documents\Master_project\Testing\Vindefjallen_testing\Arc_Vindefjellen\Arc_Vindefjellen.gdb",
        in_mosaicdataset_name=feature_class,
        coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
        num_bands=None,
        pixel_type="",
        product_definition="NONE",
        product_band_definitions="VH # #"
    )

In [28]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=r"C:\Users\trygg\Documents\Master_project\Testing\Vindefjallen_testing\Arc_Vindefjellen\Arc_Vindefjellen.gdb\aawds",
    raster_type=feature_class,
    input_path=r"C:\Users\trygg\Documents\Master_project\Data\Vindefjallen_data\S1-rasters\drive-download-20231123T194328Z-001",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="UPDATE_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="BUILD_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="ESTIMATE_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\trygg\AppData\Local\ESRI\rasterproxies\aawds"
)

<Result 'Vindefjallen_VV_MDR_test'>

In [29]:
# Set the name and data type of the new field
new_field_name = 'NewField'
new_field_type = 'DATE'  # Change this to the desired data type (e.g., 'TEXT', 'DOUBLE', 'LONG')

# Use the AddField_management tool to add the new field
arcpy.AddField_management(feature_class, field_name, new_field_type)

print(f"Field '{new_field_name}' added to '{feature_class}'.")

Field 'NewField' added to 'Vindefjallen_VV_MDR_test'.


In [30]:
timestamp_list = ['T20230108', 'T20230120', 'T20230201', 'T20230213', 'T20230225', 'T20230309']
timestamp_list.append('T20250913') # til að fá þetta til að virka með 1 eina línu á enda attrb tbl
def convert_to_datetime(date_string):
    # Extract year, month, and day from the date string
    year = int(date_string[1:5])
    month = int(date_string[5:7])
    day = int(date_string[7:9])

    # Create a datetime object
    return datetime(year, month, day)

# Convert each date string to a datetime object
date_objects = [convert_to_datetime(date_str) for date_str in timestamp_list]
print(date_objects)

[datetime.datetime(2023, 1, 8, 0, 0), datetime.datetime(2023, 1, 20, 0, 0), datetime.datetime(2023, 2, 1, 0, 0), datetime.datetime(2023, 2, 13, 0, 0), datetime.datetime(2023, 2, 25, 0, 0), datetime.datetime(2023, 3, 9, 0, 0)]


In [31]:
# Create an update cursor to iterate through the rows in the attribute table
with arcpy.da.UpdateCursor(feature_class, [field_name]) as cursor:
    for i, row in enumerate(cursor):
        # Update the field value for each row with the current date
        row[0] = date_objects[i].strftime('%d/%m/%Y')
        
        # Increment the date by 1 day for the next row
       # start_date += timedelta(days=1)
        
        cursor.updateRow(row)

print(f"Field '{field_name}' updated successfully.")

Field 'dags' updated successfully.


In [32]:
arcpy.management.BuildMultidimensionalInfo(
    in_mosaic_dataset=feature_class,
    variable_field="Tag",
    dimension_fields="dags # #",
    variable_desc_units="Dataset # #",
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'Vindefjallen_VV_MDR_test'>